In [ ]:
import numpy as np
import random
from NumfaFun import CreatePoblation,NumDBLF,create_Bin,NumDBLF
from numba.typed import List as NumbaList
from BPGen import UDLPROBLEM,PROBLEM4
from numba import types, prange,njit,deferred_type
from typing import List
from collections import OrderedDict
from numba.experimental import jitclass


In [ ]:
specff = OrderedDict()
specff['brightness'] = types.float64
specff['position'] = types.ListType(types.int64)
@jitclass(specff)
class DFF:
    def __init__(self,genome:List[int] ):
         self.brightness = 0
         self.position = genome
ff_type = deferred_type()
ff_type.define(DFF.class_type.instance_type)
@njit
def make_firefly(data:List[int]):
    return DFF(data)


In [ ]:
@njit(nogil=True)
def CalcFi(gene:List[int], boxesData:List[List[int]], container:List[int])->float:
    bin = create_Bin(NumbaList(container))
    boxesData = NumbaList(boxesData)
    gene = NumbaList(gene)
    NumDBLF(bin,gene,boxesData)
    resp = (bin.getLoadVol()/(container[0]*container[1]*container[2]))*bin.getN()
    return resp

In [ ]:
@njit
def Hamming(ff1:DFF,ff2:DFF)->int:
    count =0
    n = len(ff1.position)
    for i in np.arange(n):
        if ff1.position[i] != ff2.position[i]:
            count +=1
    return count

In [ ]:
@njit(nogil=True)
def LightInt(f1:DFF,f2:DFF,ganna:float)->float:
    d = Hamming(f1,f2)
    return f1.brightness/(1+ganna*d*d)

In [ ]:
@jitclass
class DRandMovement(object):
    def __init__(self):
        pass
    def InversionMut(self,fi_pos:List[int])->List[int]:
        n = len(fi_pos)
        step = int(n/3)
        rn = random.randint(3,n-step)
        tmp = fi_pos.copy
        tmp= tmp[0:rn]+tmp[rn:rn+step][::-1]+tmp[rn+step:n]
        return tmp
    def AlphaFF(self,fi:DFF,alpha:int)->List[List[int]]:
        newPositions = list()
        for i in np.arange(alpha):
            newPositions.append(self.InversionMut(fi.position))
        return newPositions
ranMov_type = deferred_type()
ranMov_type.define(DRandMovement.class_type.instance_type)
@njit
def instanceRanMov():
    return DRandMovement()

In [ ]:
@njit(parallel=True)
def InstanceFFn(pob:List[List[int]],boxesData:List[List[int]], container:List[int]):
    lst = []
    for i in np.arange(len(pob)):
        ff = make_firefly(NumbaList(pob[i]))
        ff.brightness = CalcFi(ff.position,boxesData,container)
        lst.append(ff)
    return lst

In [ ]:
@njit(parallel=True)
def RandomMov(firefly:DFF,alpha:int,DataBoxes:List[List[int]],BinData:List[int]):
    AlphaStep(firefly,alpha) # n2
    firefly.brightness = CalcFi(firefly.position,DataBoxes,BinData)

In [ ]:
@njit(parallel=True) #n2 
def BettaStep(f1:DFF,f2:DFF,gamma:float,hamming:int):
    n = len(f1.position)
    noneNumber = n
    res = [-1 for i in np.arange(n) ]
    visited =  [False for i in np.arange(n+1)]
    for i in np.arange(n):
        if f1.position[i] == f2.position[i]:
            res[i] = f1.position[i]
            noneNumber -=1 
            visited[f1.position[i]]=True
        else:
            visited[res[i]]=False
    betta = 1/(1+gamma*(hamming**2)) #probabilidad de que tanto ''ataree'' esto, porcentaje que se van a duplicar por cada ff
    for i in np.arange(n):
        if res[i] == -1:
            pr = random.random()
            if pr < betta:
                for j in range(n):
                    if not visited[f2.position[j]]:
                        res[i]=f2.position[j]
                        visited[f2.position[j]]=True
                        break
            else:
                for j in range(n):
                    if not visited[f1.position[j]]:
                        res[i]=f1.position[j]
                        visited[f1.position[j]]=True
                        break
    return res

In [ ]:
@njit(parallel=True)
def AlphaRandMov(firefly:DFF,alpha:int):
    movement = instanceRanMov()
    newFFs = movement.AlphaFF(firefly,alpha)
    return newFFs

In [ ]:
@njit(parallel=True)
def AlphaStep(ff:DFF,alpha:int):
    pos = ff.position.copy()
    n = len(pos)
    for i in np.arange(n):
        tmp = pos[i]
        pos[i] = round(pos[i] + alpha*(random.random()-0.5))
        if pos[i] < 1:
            pos[i] = 1
        elif pos[i] > n:
            pos[i]=n
        for j in np.arange(n):
            if i != j and pos[j]==pos[i]:
                pos[j] = tmp
                break
    ff.position = NumbaList(pos)
    

In [ ]:
@njit
def train(num_pop:int,gamma:float,Maxitr:int,DataBoxes:list[list[int]],BinData:list[int]):
    n = len(DataBoxes)
    IniPob = CreatePoblation(num_pop, DataBoxes)
    fireflyPob:List[DFF] = InstanceFFn(NumbaList(IniPob),datos,contenedor)
    for _ in np.arange(Maxitr):
        alpha = np.floor(n-((_)/Maxitr)*(n))
        fnum = len(fireflyPob)
        for i in np.arange(fnum-1):
            for j in np.arange(i+1,fnum):
                Ii = LightInt(fireflyPob[i],fireflyPob[j],gamma)
                Ij = LightInt(fireflyPob[j],fireflyPob[i],gamma) 
                if Ij < Ii:
                    fireflyPob[j].position= NumbaList(BettaStep(fireflyPob[j],fireflyPob[i],gamma,Hamming(fireflyPob[j],fireflyPob[i])))
                    RandomMov(fireflyPob[j],alpha,DataBoxes,BinData)
        RandomMov(fireflyPob[0],alpha,DataBoxes,BinData)                   
        fireflyPob.sort(key=lambda x:x.brightness,reverse=True)
    print(fireflyPob[0].brightness/n,fireflyPob[0].position)
    return fireflyPob[0]


In [ ]:
@njit(parallel=True)
def Train(Maxitr,fireflyPob,gamma):
    fnum = len(fireflyPob)
    for _ in np.arange(Maxitr):
        alpha = np.floor(n-((_)/Maxitr)*(n))
        for i in np.arange(fnum-1):
            for j in prange(i+1,fnum):
                dust = Hamming(fireflyPob[i],fireflyPob[j])
                Ii = LightInt(fireflyPob[i],fireflyPob[j],gamma)
                Ij = LightInt(fireflyPob[j],fireflyPob[i],gamma) 
                if Ij < Ii:
                    fireflyPob[j].position= NumbaList( BettaStep( fireflyPob[j], fireflyPob[i], gamma, Hamming ( fireflyPob[j] , fireflyPob[i] ) ))
                    RandomMov(fireflyPob[j],alpha,datos,contenedor)
        RandomMov(fireflyPob[0],alpha,datos,contenedor)                   
        fireflyPob.sort(key=lambda x:x.brightness,reverse=True)
    return fireflyPob[0]

In [ ]:
P4A2 = PROBLEM4[1]
contenedor = UDLPROBLEM["UDL2"]["dimension"]
datos =UDLPROBLEM["UDL2"]["24boxes"][2]

In [ ]:
contenedor =np.asanyarray( P4A2[0],dtype=np.int64)
datos =np.asanyarray(P4A2[1],dtype=np.int64)
gamma=0.0004
Maxitr = 500
n = len(datos)

In [ ]:
IniPob = CreatePoblation(50, datos)
fireflyPob = InstanceFFn(NumbaList(IniPob),NumbaList(datos),NumbaList(contenedor))
Train(1,NumbaList(fireflyPob),gamma)

In [ ]:
bestff = Train(500,fireflyPob,gamma)

In [ ]:
from Contenedor import Bin
from PackingH import DBLF2
bin = Bin(contenedor,len(datos))
boxesData = datos
DBLF2(bin,bestff.position,boxesData)
print(bestff.position, bestff.brightness)
print([item.id for item in bin.getBoxes()])
print(bin._loaded_volume/np.prod(contenedor), bin.getN(), len(boxesData))

In [ ]:
from DataLecture import ExpotarJSON

ExpotarJSON("C:/Users/nicoo/BPP/Assets/bbp.json",bin.getBoxes(),bin.dimensions)